<a href="https://colab.research.google.com/github/atg2431-alpha/pytorch/blob/main/training_pipeline_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import torch
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')

In [3]:
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [4]:
df.drop(columns=['Unnamed: 32','id'], inplace=True)
df.shape

(569, 31)

In [5]:
x_train, x_test, y_train, y_test = train_test_split(df.drop(columns=['diagnosis']), df['diagnosis'], test_size=0.2, random_state=42)


In [6]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [7]:
x_train

array([[-1.44075296, -0.43531947, -1.36208497, ...,  0.9320124 ,
         2.09724217,  1.88645014],
       [ 1.97409619,  1.73302577,  2.09167167, ...,  2.6989469 ,
         1.89116053,  2.49783848],
       [-1.39998202, -1.24962228, -1.34520926, ..., -0.97023893,
         0.59760192,  0.0578942 ],
       ...,
       [ 0.04880192, -0.55500086, -0.06512547, ..., -1.23903365,
        -0.70863864, -1.27145475],
       [-0.03896885,  0.10207345, -0.03137406, ...,  1.05001236,
         0.43432185,  1.21336207],
       [-0.54860557,  0.31327591, -0.60350155, ..., -0.61102866,
        -0.3345212 , -0.84628745]])

In [8]:
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

In [9]:
x_train_tensor = torch.from_numpy(x_train)
x_test_tensor = torch.from_numpy(x_test)
y_train_tensor = torch.from_numpy(y_train)
y_test_tensor = torch.from_numpy(y_test)

In [10]:
x_train_tensor.shape

torch.Size([455, 30])

In [18]:
class MySimpleNN():
  def __init__(self, x):
    self.weights = torch.rand(x.shape[1], dtype=torch.float64, requires_grad = True)
    self.bias = torch.zeros(1,dtype = torch.float64, requires_grad= True)

  def forward(self,x):
    z = torch.matmul(x, self.weights) + self.bias
    y_pred = torch.sigmoid(z)
    return y_pred

  def loss_function(self,y_pred,y):
    epsilon = 1e-7
    y_pred = torch.clamp(y_pred, epsilon, 1 - epsilon)

    loss = -(y * torch.log(y_pred) + (1-y)*torch.log(1-y_pred)).mean()
    return loss


In [19]:
learning_rate = 0.01
epochs = 25

In [21]:
#create model
model = MySimpleNN(x_train_tensor)

#forward pass
for epoch in range(epochs):
  y_pred = model.forward(x_train_tensor)

  #loss calculate
  loss = model.loss_function(y_pred,y_train_tensor)

  #backword pass
  loss.backward()

  #update params
  with torch.no_grad():
    model.weights -= learning_rate * model.weights.grad
    model.bias -= learning_rate * model.bias.grad

  #zero grad
  model.weights.grad.zero_()
  model.bias.grad.zero_()

  print(f'Epoch: {epoch+1}, Loss: {loss.item()}')

Epoch: 1, Loss: 0.7682570297088424
Epoch: 2, Loss: 0.7662508926044773
Epoch: 3, Loss: 0.7642498777022727
Epoch: 4, Loss: 0.7622539691892475
Epoch: 5, Loss: 0.7602631512525923
Epoch: 6, Loss: 0.7582774080836302
Epoch: 7, Loss: 0.7562967238851275
Epoch: 8, Loss: 0.7543210828752399
Epoch: 9, Loss: 0.7523504692926286
Epoch: 10, Loss: 0.7503848674035093
Epoch: 11, Loss: 0.748424261504781
Epoch: 12, Loss: 0.7464686359304751
Epoch: 13, Loss: 0.7445179750572604
Epoch: 14, Loss: 0.7425722633075518
Epoch: 15, Loss: 0.7406314851573298
Epoch: 16, Loss: 0.7386956251383403
Epoch: 17, Loss: 0.7367646678448276
Epoch: 18, Loss: 0.7348385979379045
Epoch: 19, Loss: 0.7329174001495717
Epoch: 20, Loss: 0.7310010592881181
Epoch: 21, Loss: 0.7290895602423068
Epoch: 22, Loss: 0.727182887985612
Epoch: 23, Loss: 0.7252810275818027
Epoch: 24, Loss: 0.7233839641867703
Epoch: 25, Loss: 0.7214916830549493


In [26]:
#evaluation
with torch.no_grad():
  y_pred = model.forward(x_test_tensor)
  y_pred = (y_pred > 0.5).float()
  accuracy = (y_pred == y_test_tensor).float().mean()
  print(f'Accuracy: {accuracy.item()}')
print(y_pred)

Accuracy: 0.859649121761322
tensor([0., 1., 1., 1., 0., 1., 1., 1., 1., 0., 0., 1., 0., 1., 0., 1., 0., 0.,
        0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 0.,
        0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 1., 1., 1., 1.,
        1., 0., 1., 1., 1., 0., 0., 1., 1., 0., 0., 0., 1., 1., 0., 0., 1., 1.,
        0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 1., 0., 1., 1., 1., 0., 0.,
        0., 0., 0., 0., 0., 0., 1., 1., 0., 1., 1., 0., 1., 1., 0., 0., 1., 1.,
        0., 0., 1., 0., 1., 1.])
